<center>
  
# TABSYN: Tabular Data Synthesis with Diffusion Models

</center>

Two challenges regarding the extention of diffusion models to tabular data are:
1. **Diverse data types:** a single table can have different columns each containing data of different types, including numerical, categorical, text, etc.
2. **Varied distributions:** the distribution of data under different columns in a single table varry widely from column to column.

**TabSyn** addresses these challenges by introducing a latent space where tabular data of all columns are jointly represented. It then proceedes to train a diffusion model on the latent representations.
This tactic allows TabSyn to:
1. Train a single diffusion model for all data types in the dataset (i.e. Generality).
2. Optimize the distribution of latent embeddings to facilitate training of the subsequent diffusion model, thus generating higher quality synthetic data (i.e. Quality).
3. Require much fewer reverse steps during training of the diffusion model, and synthesize data faster (i.e. Speed).

In this notebook, we review and implement the TabSyn model. The notebook is organized as follows:

1. [Imports and Setup]()


2. [Default Dataset]()
    
    
3. [TabSyn Algorithm]()
    
    3.1. [Load Config]()
    
    3.2. [Make Dataset]()
    
    3.3. [Instantiate Model]()
    
    3.4. [Train Model]()
        
    3.5. [Load Pretrained Model]()
    
    3.6. [Sample Data]()
    
    3.7. [Review Synthetic Data]()


# Imports and Setup

In this section, we import all necessary libraries and modules required for setting up the environment.
Most of the libraries we need to implement TabSyn are the same as TabDDPM.
We also specify `NAME_URL_DICT_UCI`, `DATA_NAME`, `DATA_DIR` and other paths as in TabDDPM's implementation.


In [1]:
import os
import src
import json
import pandas as pd
from pprint import pprint
import os

import torch
from torch.utils.data import DataLoader

from scripts.download_dataset import download_from_uci
from scripts.process_dataset import process_data

from src.data import preprocess, TabularDataset
from src.baselines.tabsyn.pipeline import TabSyn

from src.util import visualize_default


NAME_URL_DICT_UCI = {
    "adult": "https://archive.ics.uci.edu/static/public/2/adult.zip",
    "default": "https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip",
    "magic": "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip",
    "shoppers": "https://archive.ics.uci.edu/static/public/468/online+shoppers+purchasing+intention+dataset.zip",
    "beijing": "https://archive.ics.uci.edu/static/public/381/beijing+pm2+5+data.zip",
    "news": "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip",
}

# For shared directory you can change it to "/projects/diffusion_bootcamp/data/tabular"
DATA_DIR = "data/"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
SYNTH_DATA_DIR = os.path.join(DATA_DIR, "synthetic_data")
DATA_NAME = "banking"

MODEL_PATH = "models/tabsyn"

# Default Dataset

In this section, we will download the **Default of Credit Card Clients** dataset from the UCI repository and load it into a pandas DataFrame. This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. We will use this dataset to demonstrate the TabSyn method.
For more explanation of different steps in this section, please refer to TabDDPM's notebook.

In [2]:
# download data
# download_from_uci(DATA_NAME, RAW_DATA_DIR, NAME_URL_DICT_UCI)

# process data
INFO_DIR = "data_info"
process_data(DATA_NAME, INFO_DIR, DATA_DIR)

# review data
df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "train.csv"))
df.head(10)

data_info/banking.json
45211
Processing and Saving banking Successfully!
Dataset Name: banking
Total Size: 45211
Train Size: 40689
Test Size: 4522
Number of Numerical Columns: 7
Number of Categorical Columns: 10


age            job   marital  education default  balance housing loan  \
0  50.0  self-employed   married  secondary      no   2137.0      no   no   
1  43.0    blue-collar  divorced    primary      no    -97.0     yes  yes   
2  33.0     management   married   tertiary      no    584.0     yes   no   
3  34.0         admin.   married  secondary      no   2984.0     yes   no   
4  44.0       services   married  secondary      no   1422.0     yes   no   
5  77.0        retired   married   tertiary      no   1047.0      no   no   
6  75.0        retired   married    primary      no   2970.0      no   no   
7  42.0     management   married   tertiary      no   1340.0     yes   no   
8  36.0         admin.  divorced   tertiary      no   4708.0      no   no   
9  61.0   entrepreneur  divorced  secondary      no    369.0      no   no   

    contact   day month  duration  campaign  pdays  previous poutcome    y  
0  cellular   3.0   jul      64.0       1.0   -1.0       0.0  unknown   no  
1  cellular  31.0   jul     747.0       4.0   -1.0       0.0  unknown   no  
2  cellular  25.0   aug     508.0       5.0   -1.0       0.0  unknown   no  
3  cellular  20.0   apr      11.0       3.0   -1.0       0.0  unknown   no  
4   unknown  20.0   may     129.0       8.0   -1.0       0.0  unknown   no  
5  cellular  30.0   jun     108.0       5.0   -1.0       0.0  unknown   no  
6  cellular  12.0   aug     641.0       1.0   -1.0       0.0  unknown  yes  
7   unknown  14.0   may     299.0       1.0   -1.0       0.0  unknown   no  
8  cellular  15.0   jun     104.0       1.0   -1.0       0.0  unknown   no  
9   unknown  18.0   jun      82.0       1.0   -1.0       0.0  unknown   no

In [3]:
# review json file and its contents
with open(f"{PROCESSED_DATA_DIR}/{DATA_NAME}/info.json", "r") as file:
    data_info = json.load(file)
pprint(data_info)

{'cat_col_idx': [1, 2, 3, 4, 6, 7, 8, 10, 15],
 'column_info': {'0': {},
                 '1': {},
                 '10': {},
                 '11': {},
                 '12': {},
                 '13': {},
                 '14': {},
                 '15': {},
                 '16': {},
                 '2': {},
                 '3': {},
                 '4': {},
                 '5': {},
                 '6': {},
                 '7': {},
                 '8': {},
                 '9': {},
                 'categorizes': ['yes', 'no'],
                 'max': 275.0,
                 'min': 0.0,
                 'type': 'categorical'},
 'column_names': ['age',
                  'job',
                  'marital',
                  'education',
                  'default',
                  'balance',
                  'housing',
                  'loan',
                  'contact',
                  'day',
                  'month',
                  'duration',
                  'cam

# TabSyn Algorithm

In this section, we will describe the design of TabSyn as well as its main hyperparameters loaded through config, which affect the model’s effectiveness. 

**TabSyn** consists of two parts:
1. A *variational auto-encoder (VAE)* which learns a joint representation space for the given tabular data.
2. A *Diffusion model* which learns the distribution of data in the joint representation space.

The figure below shows a diagram of the TabSyn model.

<p align="center">
<img src="figures/tabsyn.jpg" width="1000"/>
</p>

**VAE**

The left-side of the figure shows the VAE which operates in the original data space. The VAE itself consists of two parts: an encoder and a decoder. It also contains the corresponding tokenizer and detokenizer.
Each row of the input tabular data ($\pmb{x}$) is tokenized, then embedded by a transformer. Another transformer decodes the embeddings and a detokenizer reconstructs the table ($\pmb{\tilde{x}}$). The VAE is trained by minimizing the reconstruction loss between $\pmb{x}$ and $\pmb{\tilde{x}}$.

After the VAE is fully trained, the whole data ($\pmb{x}$) is tokenized and embedded. The embedding of each row is flattened to form a 1-dimensional vector $\pmb{z}$.
These 1-dimensional embeddings for all rows are stored on disk, and will later be used to train the diffusion model.

**Diffusion**

The right-side of the figure shows the diffusion model which operates in the latent representation space; in other words, it only *sees* the embeddings obtained by the VAE, not the original tabular data.
The diffusion model can be similarly divided into two parts: a forward process, and a reverse process.

The forward process receives the embedded data points. A single data point is denoted by $\pmb{z_0}$ in the figure. Gaussian noise is incrementally added to the embeddings in numerous incremental steps during the forward process. The number of the steps is denoted by $T$ in the figure. $T$ should be high enough that the distribution of embeddings at step $t=T$ is essentially a standard Gaussian distribution; in other words, the signal-to-noise ratio is practically zero.

The reverse process, on the other hand, learns to *predict* an earlier-step embedding (e.g. $\pmb{z_{t-\Delta t}}$) from a later-step embedding (e.g. $\pmb{z_t}$) via a neural network.

After the diffusion model is fully trained, the reverse process can estimate the data distribution at step $t=0$ if it receives a standard Gaussian distribution at step $t=T$. New data points can be synthesized by sampling from this estimated distribution.


## Load Config

In this section, we will load the configuration file that contains the hyperparameters for the TabSyn model. 

In [4]:
config_path = os.path.join("src/baselines/tabsyn/configs", f"{DATA_NAME}.toml")
# config_path = os.path.join("src/baselines/tabsyn/configs", f"default.toml")
raw_config = src.load_config(config_path)

pprint(raw_config)

{'impute': {'N': 20,
            'SIGMA_MAX': 80,
            'SIGMA_MIN': 0.002,
            'S_churn': 1,
            'S_max': inf,
            'S_min': 0,
            'S_noise': 1,
            'num_steps': 50,
            'num_trials': 50,
            'rho': 7},
 'loss_params': {'lambd': 0.7, 'max_beta': 0.01, 'min_beta': 1e-05},
 'model_params': {'d_token': 4, 'factor': 32, 'n_head': 1, 'num_layers': 2},
 'task_type': 'binclass',
 'train': {'diffusion': {'batch_size': 4096,
                         'num_dataset_workers': 4,
                         'num_epochs': 10001},
           'optim': {'diffusion': {'factor': 0.9,
                                   'lr': 0.001,
                                   'patience': 20,
                                   'weight_decay': 0},
                     'vae': {'factor': 0.95,
                             'lr': 0.001,
                             'patience': 10,
                             'weight_decay': 0}},
           'vae': {'batch_size': 

The configuration file is a TOML file that contains the following hyperparameters:

1. **model_params:** specifies the structure of the transformers (both encoder and decoder) in the VAE model, including number of transformer layers, number of self-attnetion heads and token dimension.

2. **transforms:** specifies the transformations and preprocessing of the data before tokenization, such as cleaning, normalization, and encoding.
    - For preprocessing numerical features, we use the gaussian quantile transformation and replace the NaN values with mean of each row.
    - For categorical features, we use the one-hot encoding method. NaN values are left unchanged, but we have the option to replace them. We have the option to drop the values that appear with less than a given minimum frequency under each column. Furthermore, we have the option to add an extra encoding step for categorical features during tokenization.

3. **train.vae:** specifies training parameters of the VAE, including batch size, number of epochs, and number of dataset workers.

4. **train.diffusion:** specifies the same training parameters as above for the diffusion model.

5. **train.optim.vae:** specifies the parameters of the *Adam* optimizer and the `ReduceLROnPlateau` learning rate scheduler used to train the VAE. Optimizer parameters include initial learning rate and weight decay. LR scheduler parameters includer `factor` and `patience`.

6. **train.optim.diffusion:** specifies the same parameters as above for the diffusion model.

7. **loss_params:** specifies parameters of the loss function used to train the VAE including `max_beta`, `min_beta` and `lambd`.

$\beta$ is the coefficient of the KL divergence term in the VAE loss formula,

$\mathcal{L}_{vae} = \mathcal{L}_{mse} + \mathcal{L}_{ce} + \beta \mathcal{L}_{kl}$
.

Parameters `max_beta` and `min_beta` determine the range of $\beta$. $\beta$ is first set to `max_beta`. If the loss stops decreasing for a certain number of epochs (e.g. $10$ epochs), then at the end of each epoch after that (e.g. epoch $11$, $12$, etc.) $\beta$ is decreased by a factor of `lambd`,
$\beta_{new} = \lambda \beta_{curr}$,
until it reaches `beta_min`.


## Make Dataset

In this section, we pre-process the data and make a dataset object.

First, we determine transformations needed for the dataset, such as normalization and cleaning, in `transforms`. Next, using `preprocess` function we load the data from disk in arrays that contain both training and test data (`X_num` and `X_cat`), as well as the number of categories for each categorical feature (`categories`) and the number of numerical features (`d_numerical`).

We then separate the train and test data in different arrays and convert them to Pytorch tensors.
We create a dataset object (`TabularDataset`) with the train data. `TabularDataset` is a simple module which returns the tokens of a single row at a time. Each row constiutes a single data sample in TabSyn. Afterwards, we create a Dataloader for the train data using the `batch_size` and `num_workers` specified in config.

In contrast, we keep the test data as tensors (`X_test_num` and `X_test_cat`). If a GPU is available, we move these tensors to GPU so that they can be accessed by the model later on.

In [5]:
# preprocess data
X_num, X_cat, categories, d_numerical = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                   transforms = raw_config["transforms"],
                                                   task_type = raw_config["task_type"])

# separate train and test data
X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

# convert to float tensor
X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# create dataset module
train_data = TabularDataset(X_train_num.float(), X_train_cat)

# move test data to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_num = X_test_num.float().to(device)
X_test_cat = X_test_cat.to(device)

# create train dataloader
train_loader = DataLoader(
    train_data,
    batch_size = raw_config["train"]["vae"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["vae"]["num_dataset_workers"],
)

No NaNs in numerical features, skipping


## Instantiate Model

Next, we instantiate the model using the `TabSyn` class. `TabSyn` class takes the following arguments:

1. `train_loader`: dataloader for train data.
2. `X_test_num`: numerical features of the test data.
3. `X_test_cat`: categorical features of the train data.
4. `num_numerical_features`: number of numerical features in the dataset.
5. `num_classes`: number of classes (i.e. categories) of each categorical feature in the dataset.
6. `device`: the device on which the model and data exist, either "cpu" or "cuda".

In [6]:
tabsyn = TabSyn(train_loader,
                X_test_num, X_test_cat,
                num_numerical_features = d_numerical,
                num_classes = categories,
                device = device)

`TabSyn` class has the tools to instantiate VAE and diffusion models, train both, and sample from the trained diffusion model.
We will demonstrate how to use these tools in the following sections.

## Train Model


The VAE and the diffusion model are trained independently. The following subsections explain each training process.


### A. Train VAE

First, we need to instantiate the VAE using the `instantiate_vae` method. This method takes the VAE model hyperparameters, optimizer and lr scheduler parameters from config, and instantiates them.

In [7]:
# instantiate VAE model for training
tabsyn.instantiate_vae(**raw_config["model_params"], optim_params = raw_config["train"]["optim"]["vae"])

Successfully instantiated VAE model.


Now that we have instantiated the VAE, we can train it using the `train_vae` function.
This function receives the loss hyperparameters and number of epochs from the config.
Moreover, it recieves `save_path` which is the directory where trained model checkpoints will be saved.

In [9]:
# os.makedirs(f"{MODEL_PATH}/{DATA_NAME}/vae")
tabsyn.train_vae(**raw_config["loss_params"],
                 num_epochs = raw_config["train"]["vae"]["num_epochs"],
                 save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

Epoch 1/4000: 100%|██████████| 10/10 [00:01<00:00,  5.16it/s]


epoch: 0, beta = 0.010000, Train MSE: 7.696165, Train CE:1.345421, Train KL:0.651864, Val MSE:5.995673, Val CE:1.280255, Train ACC:0.411608, Val ACC:0.427178


Epoch 2/4000: 100%|██████████| 10/10 [00:01<00:00,  9.79it/s]


epoch: 1, beta = 0.010000, Train MSE: 4.864534, Train CE:1.261732, Train KL:0.938304, Val MSE:3.755433, Val CE:1.174766, Train ACC:0.517490, Val ACC:0.539009


Epoch 3/4000: 100%|██████████| 10/10 [00:01<00:00,  9.89it/s]


...

epoch: 3997, beta = 0.000008, Train MSE: 0.000007, Train CE:0.000000, Train KL:3.440159, Val MSE:0.000008, Val CE:0.000000, Train ACC:1.000000, Val ACC:1.000000


Epoch 3999/4000: 100%|██████████| 10/10 [00:01<00:00,  9.82it/s]


epoch: 3998, beta = 0.000008, Train MSE: 0.000007, Train CE:0.000000, Train KL:3.440137, Val MSE:0.000008, Val CE:0.000000, Train ACC:1.000000, Val ACC:1.000000


Epoch 4000/4000: 100%|██████████| 10/10 [00:01<00:00,  9.42it/s]


epoch: 3999, beta = 0.000008, Train MSE: 0.000007, Train CE:0.000000, Train KL:3.440121, Val MSE:0.000008, Val CE:0.000000, Train ACC:1.000000, Val ACC:1.000000
Training time: 74.5365 mins
Successfully trained and saved the VAE model!


After training the VAE, we embed the training data with the trained encoder and store the embeddings in a direcotry specified by `vae_ckpt_dir`.

In [10]:
# embed all inputs in the latent space
tabsyn.save_vae_embeddings(X_train_num, X_train_cat,
                           vae_ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

Successfully saved pretrained embeddings on disk!


### B. Train Diffusion Model

Now that we have stored the training data embeddings, we need to load and prepare them for the diffusion model.
We load the embeddings using `load_vae_embeddings`. We normalize the embeddings by subtracting the mean and dividing by the standard deviation. Then, we create a Dataloader with the specified `batch_size` and `num_workers` from the config.

In [11]:
# load latent space embeddings
train_z, _ = tabsyn.load_latent_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))  # train_z dim: B x in_dim

# normalize embeddings
mean, std = train_z.mean(0), train_z.std(0)
train_z = (train_z - mean) / std
latent_train_data = train_z

# create data loader
latent_train_loader = DataLoader(
    latent_train_data,
    batch_size = raw_config["train"]["diffusion"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["diffusion"]["num_dataset_workers"],
)

Now that the data is ready, we instantiate the diffusion model with `instantiate_diffusion`. The input dimension and hidden dimention of the diffusion model is determined by the dimension of the embeddings. 
Moreover, we instantiate the optimizer and lr scheduler using hyperparameters from config.

In [12]:
# instantiate diffusion model for training
tabsyn.instantiate_diffusion(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = raw_config["train"]["optim"]["diffusion"])

MLPDiffusion(
  (proj): Linear(in_features=68, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=68, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10633284
Successfully instantiated diffusion model.


We train the diffusion model with `train_diffusion` function.
This function takes the following arguements:
1. `latent_train_loader`: dataloader for the latent representations which are used to train the diffusion model.
2. `num_epochs`: number of training epochs.
3. `ckpt_path`: directory where the model checkpoints will be stored.

In [ ]:
# os.makedirs(f"{MODEL_PATH}/{DATA_NAME}")
# train diffusion model
tabsyn.train_diffusion(latent_train_loader,
                       num_epochs = raw_config["train"]["diffusion"]["num_epochs"],
                       ckpt_path = os.path.join(MODEL_PATH, DATA_NAME))

Epoch 152/10001: 100%|██████████| 10/10 [00:01<00:00,  9.29it/s, Loss=0.391]


Epoch 00152: reducing learning rate of group 0 to 9.0000e-04.


Epoch 180/10001: 100%|██████████| 10/10 [00:01<00:00,  9.69it/s, Loss=0.397]


Epoch 00180: reducing learning rate of group 0 to 8.1000e-04.


Epoch 228/10001: 100%|██████████| 10/10 [00:01<00:00,  9.66it/s, Loss=0.372]


...

Epoch 837/10001: 100%|██████████| 10/10 [00:01<00:00,  9.44it/s, Loss=0.365]


Epoch 00837: reducing learning rate of group 0 to 9.8477e-05.


Epoch 858/10001: 100%|██████████| 10/10 [00:01<00:00,  9.55it/s, Loss=0.355]


Epoch 00858: reducing learning rate of group 0 to 8.8629e-05.


Epoch 864/10001:  90%|█████████ | 9/10 [00:00<00:00, 12.19it/s, Loss=0.346]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Load Pretrained Model

Instead of training model from scratch, we can also load weights of a pre-trained model from a given checkpoint with `load_model_state` function.
If we haven't instantiated the VAE and diffusion model beforehand, we need to instantiate them first using `instantiate_vae` and `instantiate_diffusion` methods.

In [17]:
# instantiate VAE model
tabsyn.instantiate_vae(**raw_config["model_params"], optim_params = None)

os.path.join(MODEL_PATH, DATA_NAME, "vae")
latent_embeddings_path = os.path.join(MODEL_PATH, DATA_NAME, "vae")

# latent_embeddings_path = "/projects/diffusion_bootcamp/models/tabular/tabsyn/default/vae"
# load latent embeddings of input data
train_z, token_dim = tabsyn.load_latent_embeddings(latent_embeddings_path)

# instantiate diffusion model
tabsyn.instantiate_diffusion(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = None)

# pretrained_model_path = f"/projects/diffusion_bootcamp/models/tabular/tabsyn/default"
pretrained_model_path = os.path.join(MODEL_PATH, DATA_NAME)
# load state from checkpoint
tabsyn.load_model_state(ckpt_dir = pretrained_model_path,
                        dif_ckpt_name = "model.pt")

Successfully instantiated VAE model.
MLPDiffusion(
  (proj): Linear(in_features=68, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=68, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10633284
Successfully instantiated diffusion model.
Loaded model state from models/tabsyn/banking


## Sample Data

Now that we trained the model effectively, using `sample` function we can generate synthetic data starting from compelete noise. The input of this function is as follows:

1. `train_z`: latent embeddings of the training data.
2. `info`: info about the data from the json file we reviewed at the beginning of this notebook.
3. `num_inverse`: detokenizer for numerical features.
4. `cat_inverse`: detokenizer for categorical features.
5. `save_path`: file-path where the synthetic table will be saved.

In [18]:
# load data info file
with open(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "info.json"), "r") as file:
    data_info = json.load(file)
data_info["token_dim"] = token_dim

# get inverse tokenizers
_, _, categories, d_numerical, num_inverse, cat_inverse = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                                     transforms = raw_config["transforms"],
                                                                     task_type = raw_config["task_type"],
                                                                     inverse = True)

# sample data
num_samples = train_z.shape[0]
in_dim = train_z.shape[1] 
mean_input_emb = train_z.mean(0)
tabsyn.sample(num_samples,
              in_dim,
              mean_input_emb,
              info = data_info,
              num_inverse = num_inverse,
              cat_inverse = cat_inverse,
              save_path = os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

No NaNs in numerical features, skipping
(40689, 10)
Time: 19.516742944717407
Saving sampled data to data/synthetic_data/banking/tabsyn.csv


In [16]:
df.shape

(40689, 17)

## Review Synthetic Data

Finally here, we review the synthesized data. In the following `evaluate_synthetic_data.ipynb` notebook, we will evaluate this synthesized data with respect to various metrics.

In [15]:
df = pd.read_csv(os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

# Display the first few rows of the DataFrame
df.head(10)

age            job   marital  education default       balance  \
0  24.000000         admin.    single   tertiary      no   90924.33000   
1  95.000000  self-employed   married    unknown      no  102127.00000   
2  95.000000        retired   married   tertiary      no    -394.24286   
3  95.000000     unemployed   married   tertiary      no    1381.61100   
4  18.002983        student    single  secondary      no   23024.20700   
5  18.646091     management    single   tertiary      no  102000.90000   
6  64.266760         admin.   married  secondary      no   95962.61000   
7  37.000000         admin.  divorced  secondary      no    -301.13815   
8  95.000000  self-employed   married  secondary      no  102058.52000   
9  18.001880     management    single  secondary      no     -93.07254   

  housing loan    contact   day month     duration  campaign      pdays  \
0     yes   no   cellular   6.0   may  1455.655900       2.0   -1.00000   
1      no   no   cellular  17.0   jul  4918.000000       4.0   -1.00000   
2     yes   no    unknown   7.0   may    10.000000       2.0   -1.00000   
3      no   no   cellular   8.0   aug  1558.266000       2.0   -1.00000   
4      no   no   cellular   8.0   oct   493.317440       1.0  138.91539   
5      no   no   cellular  12.0   aug    12.000000       2.0   -1.00000   
6     yes   no   cellular   4.0   may     0.565659       2.0  237.28775   
7     yes   no    unknown  12.0   may   356.065640       2.0   -1.00000   
8      no   no  telephone   7.0   jun  4918.000000       2.0   -1.00000   
9     yes   no    unknown  29.0   may    23.159323       5.0   -1.00000   

   previous poutcome    y  
0       0.0  unknown   no  
1       0.0  unknown  yes  
2       0.0  unknown   no  
3       0.0  unknown   no  
4       2.0  success  yes  
5       0.0  unknown   no  
6       0.0  failure   no  
7       0.0  unknown   no  
8       0.0  unknown  yes  
9       0.0  unknown   no

## References

**Zhang, Hengrui, et al.** "Mixed-type tabular data synthesis with score-based diffusion in latent space." *International Conference on Learning Representations (ICLR)* (2023).

**GitHub Repository:** [Amazon Science - Tabsyn](https://github.com/amazon-science/tabsyn)